In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from general_functions.datetime_helper import transform_date_to_timestamp_milliseconds
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [3]:
customer = "Autobatterienbilliger"
from_date = '2025-12-01' #pd.to_datetime(to_date) - pd.DateOffset(days=3)
to_date = '2026-02-04'
#from_date = from_date.strftime('%Y-%m-%d')
goal_name = ["checkout_completed"]#,"checkout_completed"] #"checkout_completed", "tagmanagerregistercompleted"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [4]:
conversions=pd.DataFrame()
list_dates = pd.date_range(from_date, to_date).strftime("%Y-%m-%d")
for idate, date in enumerate(list_dates):
    if idate == len(list_dates)-1:
        end_date = list_dates[-1]
    else:
        end_date = list_dates[idate+1]
    print(f"Query conversions from {date} to {end_date}")
    content={
        "created": {
                        "$gte": transform_date_to_timestamp_milliseconds(date),
                        "$lte": transform_date_to_timestamp_milliseconds(end_date),
                },
                #"name":goal_name
                    }
    temp=send_to_innkeepr_api_paginated(
        f"{url}/conversions/query",
        account_id,
        content,
        logging
    )
    temp = pd.json_normalize(temp)
    print(temp)
    if temp.empty:
        print(".... no conversions for that date")
    conversions = pd.concat([conversions, temp])
conversions

In [5]:
conversions["created"].min(), conversions["created"].max()

In [6]:
# import ast
# def return_product_name(x):
#     if x is None:
#         return x
#     if isinstance(x, str): 
#         x = ast.literal_eval(x)
#     if isinstance(x, list):
#         names = []
#         for entry in x:
#             if "name" in entry.keys():
#                 names.append(entry["name"])
#         return names
    
# conversions["properties.products.name"] = conversions["properties.products"].apply(lambda x: return_product_name(x))
# products = conversions["properties.products.name"].explode().drop_duplicates()
# products

In [7]:
conversions["signal.source"].value_counts(dropna=False)

In [8]:
conversions["date"] = pd.to_datetime(conversions["created"]).dt.date
print(conversions.shape)
vc = conversions.groupby("date")["name"].value_counts(dropna=False).reset_index()
vc.to_csv(f"DataChecks/conversions/conversions_{customer}.csv",index=False)
print(vc['count'].sum())
vc

In [9]:
conversions["properties.revenue"] = conversions["properties.revenue"].astype(float)
conversions.groupby(by=["date"])["properties.revenue"].sum().sum()

In [10]:
conversions[conversions["sessionId"]=="29652d1e-6788-4d3c-976e-0161513f6541"]

In [11]:
conversions_google = conversions[
         conversions["signal.source"] == "609ffc4578188e83a2bc2c2c"
     ]
vc_google = conversions_google.groupby("date")["name"].value_counts(dropna=False).reset_index()
print(vc_google)
conversions_google.groupby(by=["date"])["properties.revenue"].sum().sum()

In [12]:
conversions["sessionId"].nunique()

In [13]:
if "signal.source" in conversions.columns:
    temp = conversions.groupby(by=["date","name","signal.source"])["anonymousId"].nunique().reset_index()
else:
    temp = conversions.groupby(by=["date","name"])["anonymousId"].nunique().reset_index()
temp

In [14]:
conversions["signal.relates_to.treatment"].dropna().unique().tolist()

In [15]:
group = conversions.groupby("date")["signal.source"].value_counts(dropna=False).reset_index()
group

In [16]:
conversions["date"] = pd.to_datetime(conversions["created"]).dt.date
conversions["properties.revenue"] = conversions["properties.revenue"].astype(float)
group_rev = conversions.drop_duplicates(subset=["sessionId"]).groupby(by=["date","signal.source"])["properties.revenue"].sum().reset_index()
group_rev

In [17]:
conversions.groupby(by="date")["name"].count()

In [18]:
conversions.groupby(by="date")["anonymousId"].count()

In [19]:
conversions.groupby(by="date")["signal.source"].value_counts(dropna=False).reset_index()

In [20]:
conversions[(conversions["date"].astype("string")=="2025-09-25")&(conversions["signal.source"]=="609ffc4578188e83a2bc2c2c")]["sessionId"].unique()

In [21]:
conversions["signal.source"].value_counts()

In [22]:
conversions["created"].min(), conversions["created"].max()

In [23]:
conversions.drop_duplicates(subset=["name","sessionId"])["name"].value_counts(dropna=False)

In [24]:
conversions.drop_duplicates(subset=["name","anonymousId"])["name"].value_counts()

In [25]:
conversions["date"] = pd.to_datetime(conversions["created"]).dt.date
count_checkouts = conversions[conversions["name"].isin(goal_name)].drop_duplicates(subset=["name","sessionId"]).groupby("date")["name"].value_counts().reset_index().sort_values(by="date")
count_checkouts

In [26]:
# count_checkouts.plot(x="date",y="count",hue="name", kind="bar")
sns.barplot(data=count_checkouts, x="date",y="count",hue="name", palette="Set2")
plt.grid(True)
plt.xticks(rotation=90)

In [27]:
conversions["signal.source"].value_counts()

In [28]:
conversions[conversions["name"].isin(goal_name)].groupby(by=["signal.source","name"])["signal.relates_to.campaign.name"].value_counts().reset_index().sort_values(by="signal.relates_to.campaign.name")

In [29]:
conversions[conversions["name"].isin(goal_name)].groupby(by=["signal.source","name"])["signal.relates_to.treatment"].value_counts().reset_index()

In [30]:
conversions[conversions["name"].isin(goal_name)]["signal.relates_to.treatment"].unique().tolist()

In [31]:
vc_signal = conversions[conversions["name"].isin(goal_name)].groupby(by=["date","signal.source","name"])["signal.relates_to.campaign.name"].value_counts().reset_index().sort_values(by="date")
vc_signal

In [32]:
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(1,1,1)
sns.barplot(data=vc_signal[vc_signal["signal.source"]=="609ffc4578188e83a2bc2c2c"].sort_values(by="date"), x="date",y="count",hue="signal.source", palette="Set2", ax=ax)
plt.grid(True)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [33]:
conversions

In [34]:
conversions.groupby("signal.source")["properties.products"].value_counts()

In [ ]:
conversions

# Conversions by Month

In [ ]:
conversions["created"].min(), conversions["created"].max()


In [ ]:
conversions["name"].value_counts(dropna=False)

In [ ]:
conversions["month"] = pd.to_datetime(conversions["created"]).dt.month
conversions.groupby(by="month")["sessionId"].nunique()

# Match Conversion with Treatments

In [ ]:
treatments=send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"id":conversions["signal.relates_to.treatment"].dropna().unique().tolist()},
    logging
)
treatments = pd.json_normalize(treatments)
treatments.columns

In [ ]:
treatments = treatments.rename(columns={"id": "signal.relates_to.treatment", "name":"treatment_name"})

In [ ]:
conversions_with_treatments = pd.merge(conversions, treatments, how="left", on="signal.relates_to.treatment")
conversions_with_treatments

In [ ]:
conversions_with_certain_campaigns = conversions_with_treatments[conversions_with_treatments["relates_to.campaign.name"].str.contains("ASC").fillna(False)]
conversions_with_certain_campaigns = conversions_with_certain_campaigns[conversions_with_certain_campaigns["relates_to.campaign.name"].str.contains("TOFU").fillna(False)]
conversions_with_certain_campaigns["relates_to.campaign.name"].value_counts()

In [ ]:
conversions_with_certain_campaigns["signal.relates_to.treatment"].value_counts()

In [ ]:
conversions_with_certain_campaigns["date"] = pd.to_datetime(conversions_with_certain_campaigns["created_x"]).dt.date
conversions_with_certain_campaigns.groupby("date")["name"].value_counts().reset_index()

In [ ]:
conversions_with_treatments["signal.source"].value_counts()

# Match with account_signals

In [ ]:
conversions["sessionId"].dropna().unique().tolist()

In [ ]:
sessions = send_to_innkeepr_api_paginated(
    f"{url}/sessions/query",
    account_id,
    {"sessionId":conversions["sessionId"].dropna().unique().tolist()},
    logging
)
sessions = pd.json_normalize(sessions)
sessions.columns

In [ ]:
gclids = sessions["campaign.gclid"].dropna().unique().tolist()
gclids[0:10]

In [ ]:
signals = send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    {"externalId":gclids},
    logging
)

signals = pd.json_normalize(signals)
signals

In [ ]:
adAccountId = "8234087217"
mask = signals["relates_to.adGroupAd.resourceName"].astype("str").str.contains(adAccountId)
signals = signals.loc[mask, ["relates_to.adGroupAd.resourceName", "externalId"]]
signals

In [ ]:
merged = pd.merge(signals, sessions, how="left", left_on="externalId", right_on="campaign.gclid")
merged = pd.merge(merged, conversions, how="left", on="sessionId")
merged

In [ ]:
merged.groupby("date")["sessionId"].nunique().sum()


In [ ]:
merged["properties.revenue"].astype("float").sum()


# Conversions with Google Gclids

In [ ]:
check_for_anonymous_ids = ['68e60a081d063538d0dcb729', '64b562133991bf268073131d', '68e635bf1d063538d02ca781', '68e659381d063538d0644f2b', '64f8ba49f15d9306f00d9c4d', '68e6be97600b2140865b22d3', '68e5193c120f694f532d7643', '68e6d5a2600b2140868bca0e']
    
conversions[conversions["anonymousId"].isin(check_for_anonymous_ids)]

In [ ]:
sessions = conversions["sessionId"].dropna().unique().tolist()
ses_resp = send_to_innkeepr_api_paginated(
    f"{url}/sessions/query",
    account_id,
    {"sessionId":sessions},
    logging
)
ses_resp = pd.json_normalize(ses_resp)
ses_resp.columns

In [ ]:
ses_resp.groupby("sessionId")["anonymousId"].nunique()

In [ ]:
conv_with_gclid = pd.merge(conversions, ses_resp[["sessionId","campaign.gclid"]], how="left", on="sessionId")
conv_with_gclid = conv_with_gclid[(conv_with_gclid["campaign.gclid"].notna()) &(conv_with_gclid["signal.source"].isnull()==False)]
#conv_with_gclid = conv_with_gclid[conv_with_gclid["created"]>="2025-10-08"]
#conv_with_gclid = conv_with_gclid[conv_with_gclid["created"]<="2025-10-09"]
conv_with_gclid

In [ ]:
conv_with_gclid["sessionId"].nunique()

In [ ]:
conv_with_gclid["anonymousId"].nunique()

In [ ]:
profiles = send_to_innkeepr_api_paginated(
    f"{url}/profiles/query",
    account_id,
    {"anonymousId":conv_with_gclid["anonymousId"].unique().tolist()},
    logging
)
profiles = pd.json_normalize(profiles)
profiles = profiles.explode("externalIds")


In [ ]:
profiles["has_gclid"] = profiles["externalIds"].apply(lambda x: x["name"]=="gclid")
profiles["extId"] = profiles["externalIds"].apply(lambda x: x["id"])
profiles = profiles[profiles["has_gclid"]==True]
profiles

In [ ]:
print(conv_with_gclid["anonymousId"].unique().tolist())
for anomyousId in conv_with_gclid["anonymousId"].unique().tolist():
    if anomyousId not in profiles["anonymousId"].unique().tolist():
        print(anomyousId)

In [ ]:
sessions = conversions["sessionId"].dropna().unique().tolist()
ses_resp_date = send_to_innkeepr_api_paginated(
    f"{url}/sessions/query",
    account_id,
    {"created":{
        "$gte":"2025-10-08",
        "$lt":"2025-10-09"
    }},
    logging
)
ses_resp_date = pd.json_normalize(ses_resp_date)
ses_resp_date.columns

In [ ]:
ses_resp_date[ses_resp_date["sessionId"].isin(conv_with_gclid["sessionId"])]

# Ceck for E-Mail

In [ ]:
for col in conversions.columns:
    temp = conversions[conversions[col].astype("string").str.contains("@|mail")]
    if len(temp) > 0:
        print(f"Found mail in {col}: {temp}")

In [ ]:
test = [
    "_fbp",
    "_fbc",
    "email",
    "email_sha256",
    "anonymousId",
    "userAgent",
]
search_pattern = "|".join(test)

# Check Conversion for fixed session ids

In [ ]:
sessions20251116 = ['a4320a73-a57d-4aa4-baba-4419da93b2e0', '94dc8410-1f11-478b-a657-99c80ec9b903', '67c0b65e-329f-4760-bf76-f5417abcd627', 'd0a011af-080d-4455-87b1-1ed4186b36c0', 'a91496b2-c3d5-46b8-9760-c020e078eac4', 'c82ac265-4993-4043-89fc-8d829db9afb7', '96e6b69d-e312-425d-8be3-8df00785e256', '9878058b-bffa-4402-917e-d5839669609c', 'd2cba60c-2575-495d-9d8a-7a2ecd467ec6', '732fbf93-7a61-4256-82d0-9ba8c1da0113', 'f3dd6944-55cc-42f8-9c6c-1bd419f60f5f', 'ff761add-9582-48bf-ba66-ab8dc4386e16', '923e1f0d-80b8-416f-ad2d-1b9cb739c739', 'b6b2b917-ab2b-4571-8f6c-dbdefa6a609a', '6ff419c1-a591-49d4-985c-b1ee7040fdda', 'b423982d-4111-41a8-baa7-4688c60ebba1', '304a21c1-c54e-4a19-b7a2-9d3f9a52f7f7', 'b7c9eec1-f2e6-4f41-ac9d-f22ada69f5c7', 'f3ba2093-ff94-48f9-8d46-7d3e1f579225', 'c29c76d0-d393-4a58-bb7e-cd46c31e26b7', '7a043ff2-c133-474b-8bf2-bb7fb679416c', 'c1ff824a-ec01-410b-bd78-67f321ef5eeb', 'd0c0b50e-86e7-461a-bc20-f7d4d83d8697', '9e49b856-b918-4e70-9240-8173c8691142', 'a5d88e38-48a5-44c6-8636-c39dff5d68db', 'fe7952bc-7e44-4490-a219-abbb93bc121f', '937cb63c-dd4e-4a5d-bdf1-24373605af92', '789e1f1d-dbc6-4089-911e-12f7be4fbb93', '15a5fea6-b0a2-452c-bdc6-9c0d8bbe93ea', 'dd0ad31f-a228-406f-ad34-5232221cf915', '8280352d-2cba-43e9-9eca-4c383455e568', '032e132c-693c-4d28-87ee-3058f0693e5f', '8845bb1d-5ce4-4a0c-b432-d75024440ffd', '6dbfc5b4-694b-49fc-86b5-6d692483e189', 'efcc0233-9826-4a98-9607-c037bf4e7577', 'ceee6432-cfb5-4086-94a8-311f8b982f12', '44c57bdb-4e71-42dc-a36f-fcf4ffb06d7b', '39fb8196-04be-41c3-81be-81b85f4db85b', 'b4e46d4e-366d-4c25-bd20-f17d30977a28', 'b50cb196-0619-4ed5-ba7f-5a30fde3bd4d', '9b05397a-1882-410b-bbbd-6231a1e7503d', 'ed76e03e-046a-4243-8c9c-cb22426313cd', 'a54e9eb3-3cf7-4c4e-ac80-3a2b8f5acb91', 'cf2378a5-2897-4935-9dbb-5e2fccd2ed87', '93164f26-d91a-4536-8a92-f5d4a09ca915', '7c6c70b6-1049-4de3-92b7-de52c3f9d11a', 'c9c5cf37-48c8-4c82-bb5b-8f61ac102788', '2eb20083-63c5-4215-8236-d7253a503bf6', '8233c2c7-8d3e-4e9d-926e-85cce1584400', 'a9b302e2-0481-413f-945a-7b2fc9069bcf', '5d2209cb-7b15-4873-994d-400a74f2e016', '514ad864-682e-4a53-b90d-f4d2ae56839c', '4dffb2f4-afba-4be9-b375-9c3dcf5bb654', 'b76782d3-ca04-4f9b-8444-6db78c2e910d', '57111780-f50e-431b-91ac-4eba1d0246ea', '0106ab1d-92d3-4f50-aa68-60f82be36e03', 'e1483416-cc0e-4e85-af28-d82b8cf3898a', '80dfb3e1-05cf-41fd-90db-cd1ee5e093b6', '40874d10-ca57-4786-91d2-91fe09900073', 'a925e494-ef69-4164-bc67-a6623dbfd0c7', '9ec6ea6b-81d0-40a3-ab80-0b96f49490e7', 'fc256f52-f20c-4049-876e-4e5613406c10', 'a825079a-2356-4d6b-9475-32a6d288440b', '2a226185-07f4-4c11-8873-f671d9c69fa4', 'd61e4b7d-63c0-4372-9dca-eef637d3cd7e', 'a3731116-98b1-4975-8f33-409be8947601', '97110438-4d39-4a23-9f2c-7ee8d89db262', '8a671c2e-83ad-4960-9726-87dd63610abf', 'fabc16d7-e6c7-49f6-b8f5-f63ae603399b', 'aff92d2e-e7de-40a5-bc33-878a7f788513', 'fb75ef03-d9d2-4c17-ac79-1778656ad2b1', 'bea95ff2-2487-4673-bd89-e529fda2306e', 'b2d77eb8-349b-4514-9eb8-6c6b12b215d9', '13da6935-78a6-40ed-b978-bfa26a16d976', '1f41f75b-87c5-4ccd-bf9d-57ad34b6d5c9', 'f196ce31-60fc-414e-8f34-436964d6f971', 'e5cac712-f5c0-4745-ba7d-08ccb7d2c2df', 'ee1fa444-ab1e-4afc-9299-f778b619d065', 'ca8c48f7-1a6b-4be2-b516-73f837c656d1', '90f0b245-bd48-4629-9873-731afead9c92', '8e876be7-ce6a-4310-9063-cfdb8b1a1b9c', 'c9875c10-e0cf-434d-9858-748e9ad76496', '303036bb-3cd8-4f8f-ac06-0e29660ce425', '8ec54eac-6a25-4566-98d4-aae2a7ff4928', '69be24a0-d05a-4e48-a67b-75d09f385045', '800bfdea-e7f8-4a0b-8b83-fa01aa63e4e0', '6e114958-38ea-4dab-b4ba-23b6b2b15de6', '02df95e2-3aa0-4a90-9b3b-e84b52507b81', 'c1b9ba98-e773-49aa-ab9c-e500f42d8e0c', 'b8ed26d9-b301-475b-8f84-29c2c9c04c6e', '57d97e9d-6a4a-4739-99bb-d6021ccb7193', '04be6959-f0de-44b3-b4b4-aa18046f345f', 'e23d170d-38ce-495d-a404-68bc14b7dc5e', 'cfd76af9-97fe-4f25-9ee5-1f9e73b6f168', 'd21d900c-1ee8-4302-97c9-b73506892f81']
sessions20251115 = ['85df635a-6a4c-4e7c-bcb3-dad274314fe4', '28dd2e8e-80fb-49bf-866e-3886fc414022', 'c6c309fe-6c9a-4101-9f49-3a5e8cb62f05', '9ba7be2f-6787-4cd2-8d99-a03f9c8a708b', 'ddd3107e-4d4b-44a0-8b33-9f49cf3508fc', 'cf2cf0df-3f63-4395-abbc-70977ee1606d', '6d7430ee-737e-4a91-a4b4-a995973465c1', '9e8a8ffb-3942-46e3-9686-b942f9e5d4be', '9d138736-5592-480d-9f07-41e36ad74c8c', 'e9764a31-0813-456b-a5d3-eaa7c96a1915', '23d34a83-890e-4a88-adb7-c89d27e603a2', 'fec9ec64-ff5d-438d-b023-aea43bd00c54', '2ffc085b-52e9-469a-8c9e-5530ea438a20', 'a2f31800-b454-4db3-b87b-de37c1f54e5a', '8d3d4430-8cfa-48b4-ae0d-c997b79307b8', '76018621-7d1b-4a2f-897a-63329a117d66', '90b6560e-1594-4563-8481-c2aff10b848c', '37f13e97-e99f-4003-a034-5aca82815fcd', 'c37e4840-f1e7-4e85-a1c4-35604aa0305f', '84d3c551-ec05-4602-8014-601d90bbdf48', '01e00593-d4c1-4f62-b135-e90f339ac48b', 'e55fed8f-99f4-4213-9414-6ec901c9784a', '11277d7d-7057-40fe-87db-f105c22705e7', '98874ef3-0dfb-4dba-b7ca-475e67b988d8', '6f5f506b-c71c-4450-b2a8-d634e916102e', '94275e72-b138-43bd-b21a-aa7a1ba70439', 'da3d2915-4abd-467f-84a8-dc67418860dc', '22906610-4ba8-4008-8303-2af4132e2e5d', '336bf572-4846-495f-9460-7ea4c27025b1', '1bbe606b-76c1-4912-a456-7f39b3c8f21e', 'a399eca8-ce84-4810-82c9-b80672548773', 'dafb5225-749f-4e4e-9ed8-1a336beae8b6', '35e9142b-1acc-464f-b6a8-bfd49fb6a4b6', 'e8734303-495c-4611-8abb-783450ffab73', 'd427ae38-bf40-4b79-8a58-69ece82db1b3', '3d1591b0-b624-44b5-9c46-99fdf1139d78', '8a6251a0-fb9c-4a1c-bdb5-a61f6fc46a14', '3cf6f1a3-8ab6-406e-b1b5-50625cf80289', 'ea5931b0-547b-422a-9b31-a7ac75eaf85f', 'da4dda5f-0ebb-4590-afd3-ba0fea708014', '8c414af4-1926-4ef8-a5f5-27eec0419894', 'f7ae1dc4-beab-4f85-bc91-8477720f602d', '957f4ddf-fadd-4df7-9d00-63b41f12e966', '0fd77c07-12f7-49dc-9f08-d715824b99fb', '75984fa7-9cb8-4104-828a-16048a44cda3', '0ac7949d-b119-4136-a131-314997128fe3', '4d09e9a9-8441-4d79-a28d-369adde0afd2', '3b6f4a2c-90ed-4f3e-abac-f1582866d897', 'b544f1b7-c3e1-4fcb-84d2-4a8938c81053', '69a507e2-7ab1-4d09-aec3-9f65b5574be7', '126422c9-fa32-47d6-b6fa-29bcdfc368ab', '4db23212-a624-47f2-befe-b52da2a1e6aa', 'a66dca18-3846-4405-bfb3-65c76f252752', 'c60f95af-d8ff-4e64-9bad-2a11b06e862a', 'c63b8082-a85f-4230-b4cd-17527fd83eb7', 'fd56a6fe-5797-4c74-97ee-f6a7df8a7604', '093f0407-ede2-4656-b685-16bf95087c66', 'afd8306e-b1d9-4e3b-ab9b-1a2bd8c76b38', 'c83146bf-56e3-4b62-be30-f48b8e72c98f', 'ee690907-919f-4575-a291-61f01a80cda6', '77dc9120-1255-4b8d-ac4b-78120262d822', 'b045e3df-dfe0-43c0-b28e-2622793bdf4b', '0fcfd1f8-c848-4488-bab6-d176398c97d1', '8ac829bd-8f13-4fde-a6b7-a900e1444c9f', '25d74b0e-5c9d-4a72-875c-7f0f506f8e31', '9e31dd0e-c520-48f9-afde-0a10c1ce11c7', 'b727e3fb-a76b-4b63-93b5-126b5842d60f', '39dff5b4-770c-426b-8c0d-c118f9e0357c', '3a3c065f-b732-4387-8ce6-e73d44dc8cb1', '6b2096a8-8c70-4672-b11c-980d0dc440e5', 'f89324c1-45fe-42df-a654-d444fb37f7d1', 'df0733d7-dde3-4e51-b38f-d98e9ad383c3', '850d34c2-29a0-43db-b114-326a4c386475', 'ab735902-0e8a-428c-adc8-9588efd5019c', '241c72c0-1ca6-426e-a4f0-0df3f203526e', '2ff26515-6aac-44c7-b530-86f2b0d758b7', '071d1e85-3d47-4355-8f7b-ff101e53125a', 'd73c2495-3200-4e3c-beed-3a41e36521c7', '0b7d389a-d5a8-4448-863c-70ced259fc3e', 'ac2ee8b9-2a3f-4d6c-b538-0fd21b533e04', '762884f3-dd2c-4434-8cf5-35d59b9809d8', 'bef4cef0-6c78-4e09-82a2-12de34cd7c50', 'b59fde48-25a8-4041-9107-607c8a1e0ef8', 'f816cd0b-807b-43ef-ae56-178a91c2bb55']
conversions[conversions["sessionId"].astype("string").isin(sessions20251115)]

In [ ]:
list_anonymousId = ['689b9cae97b976aeeffb578f', '690f26c03517fb7408a98adc', '6908a8788b306ae85333ac83', '68ff873e857b19e246e05a67', '68890d25eafeb4bf3b3010b8', '68fbf27b857b19e24610eea1', '689a42e697b976aeef8956d6', '68a1fdfe97b976aeef04629f', '690ca1bbf57ecede23b4f92d', '69021a03857b19e2469e8576', '6894588fa7f5487f82e3b1c6', '68e92d88b236f6791074b62e', '68ed5d88b236f67910ac36b8', '68fef138857b19e2462492b8', '68b1b0e70e302bdecd43d078', '68f8b57b2bc7bd299ba92e18', '68d031e52a849e57210222ce', '68e07144120f694f53094864', '68b2bb420e302bdecdab540d', '688f0a266a367afb622819d8', '68f638012bc7bd299bcd4adb', '68c9450ad594535196daecee', '68a5b1aeb6ad893ce3e5a35f', '688b8166eafeb4bf3bd66327', '68b0be2e0e302bdecde9719e', '690d2f14f57ecede2313c717', '689dcada97b976aeefaf6215', '68b87c785287866a2c537787', '68ce74787837f1e5c866e268', '68d9792d6c601ba6361fa51c', '691982ff3517fb74088e72ea', '6906484b8b306ae8531fee45', '689732d3a7f5487f82ca6806', '688a5fd2eafeb4bf3b874d21', '68ad1357d161e24cb7d5b246', '68f280fc2bc7bd299be82c33', '68a0986897b976aeef974f13', '689658a4a7f5487f82983004', '68f8430d2bc7bd299b025161', '68f3bf042bc7bd299b32726c', '688e4d596a367afb62ff6a8f', '689f05d197b976aeef1bea65', '68f8924a2bc7bd299b619703', '68e74c31600b214086be274e', '68fdebf3857b19e246258583', '690c552d8b306ae853279bb7', '6889fe3ceafeb4bf3b68399f', '68f798572bc7bd299bc86502', '690342f5857b19e246d0f4d7', '68dff6fb120f694f53906a19', '68c05cd77bd2ec4485831bbc', '690107f9857b19e2467eaa26', '689de76197b976aeefbfa74e', '68c4473c52d0b4ef38079c72', '68da38276c601ba6366ef5d4', '68f66f172bc7bd299b559bd5', '688e21506a367afb62efa698', '689c9c7d97b976aeef44d1bf', '68f777302bc7bd299b7cdc0b', '68e12fe4120f694f53daa249', '68a4a23db6ad893ce3929053', '68fa37be857b19e246645aab', '689e0a7e97b976aeefd1448c', '68c58fc352d0b4ef3885a405', '688bc5d1eafeb4bf3bef4fb3', '68bfd93e7bd2ec448533ce30', '68eeafe0a7adbdaa70e8837c', '68e8d3e06551590395f31420', '688b0ae1eafeb4bf3ba51dd0', '68de5986120f694f53c9f666', '68e0f61c120f694f53812a92', '68fb6549857b19e24622a778', '6903bcb38b306ae8532b2a12', '68ff0a9e857b19e2463e1a8f', '68da0b216c601ba63657209b', '68c316ad52d0b4ef3880edf6', '6901b39a857b19e24697bf19', '69034ba4857b19e246ec5f5d', '68a217f597b976aeef0fbdb3', '688af480eafeb4bf3ba02f45', '68f6309b2bc7bd299baf32b0', '68f7eec12bc7bd299bc09449', '688e44f76a367afb62fc7d58', '689312a1a7f5487f827dacd5', '689a1d4b97b976aeef79e870', '6903601a8b306ae8539cab88', '6905911b8b306ae85330c68a', '68a6108db6ad893ce30d5cd0', '68ea8e36b236f679102fec3c', '68c255677bd2ec4485657ab4', '690f5a613517fb7408a646de', '68dd8075120f694f531851fc', '6900aef2857b19e2469948b0', '68891903eafeb4bf3b35030e', '68c07e717bd2ec44859a1c58']
conversions[conversions["anonymousId"].astype("string").isin(sessions)]